## Mpls Marriott Hotel Simulations
8760 rows per simulation<br>

## Preprocess

1. Generate zone with facade glazing systems in COMFEN.

2. Retrieve IDFs for each scenario from the COMFEN5 Results folder. Have to do each of the 3 sections of the building as its own sqlite file and each project by orientation separately. Then IDF files are copied after each simulation. These files will be used in the Jupyter Notebook.

3. Convert from E+ 8.1 to 9.5 using the .lst format.

4. Use E+ template file to change the HVAC system. See Word doc that describes the process. Process all IDFs.

5. If changing the angle of due north, pay attention that COMFEN has clockwise going negative. For this project E=90, NW=-120, SW=-60

6. Rerun batch sims in EnergyPlus v9.5.

7. Add "Scenario", "Elevation, and "GlazingSystem" to the results CSV files.

In [1]:
# Dependencies
import pandas as pd
import os
import numpy as np
from pathlib import Path
from datetime import datetime

## Energy Conversions

As per E* Documentation<br>

Joules --> kBtu (multiply by .00000094781712)<br>
Joules --> kWh (multiply by .000000277777778) or (divide by 3600000)<br>
Joules --> therms (multiply by .0000000094781712<br>

<div>
    <img src="static/images/elect_conv.png" width="400"/>
     <img src="static/images/gas_conv.png" width="400"/>
</div

## Energy Costs and Emission Factors

http://seeatcalc.gastechnology.org/ResidentialBuildings.aspx

In [2]:
# Costs (elec $/kWh, gas $/therm), change values per city
# State Average
# comgas_cost = 0.60
# comelec_cost = 0.1043

# Provided by client
comgas_cost = 0.62915
comelec_cost = 0.127249

In [3]:
# Source Energy Conversion Factors (same for res or comm), change values per city
gas_source = 1.09
elec_source = 2.65

In [4]:
# Carbon equivalent factors (same for res or comm), change values per city

#Electricity (lb/MMBtu) 323.34 (lb/kbtu).32334 ####
#Natural Gas (Building Used, lb/MMBtu) 145.66 (lb/kBtu .14566) ####

gas_co2 = 0.14566
elec_co2 = 0.2964

In [5]:
# Zone size for EUI and cost/sf

sf = 210

In [6]:
# Lighting energy reduction, assumption that LEDs are being used. 
# Use this multiplier if reductions are not in the COMFEN model

lighting_redux = 1

In [7]:
# # For peak demand calculations (.0002931 kW = 1 Btu).

# kw = 0.0002931

---

## Combine CSV Files in Directory 

In [8]:
# Create variable for files in directory
files = [f for f in os.listdir("data/msp2/") if f.endswith(".csv")]

# files

In [9]:
# https://stackoverflow.com/questions/63886787/how-to-create-a-dataframe-from-multiple-csv-files
# Initialize and empty dataframe
all_ComElec = pd.DataFrame()

# Iterate through files and contents, then concatenate the data into the dataframe
for file in files:
    df = pd.read_csv("data/msp2/" + file)
    all_ComElec = pd.concat([all_ComElec, df])
    
# all_ComElec

# 78840  rows, 55 columns

In [10]:
# list(all_ComElec.columns.values)

In [11]:
# Get wanted columns
all_energy = all_ComElec[["Scenario",
               "Date/Time",
                "Elevation",
               "GlazingSystem",
              "COMFENZONE:Zone Lights Electricity Energy [J](Hourly)",
              "VAV SYS 1:Air System Fan Electricity Energy [J](Hourly) ",
              "MAIN CHILLER:Chiller Electricity Energy [J](Hourly)",
             "MAIN BOILER:Boiler NaturalGas Energy [J](Hourly)"]]

# all_energy

In [12]:
all_energy = all_energy.dropna().reset_index(drop=True)

all_energy

,Scenario,Date/Time,Elevation,GlazingSystem,COMFENZONE:Zone Lights Electricity Energy [J](Hourly),VAV SYS 1:Air System Fan Electricity Energy [J](Hourly),MAIN CHILLER:Chiller Electricity Energy [J](Hourly),MAIN BOILER:Boiler NaturalGas Energy [J](Hourly)
0,Eclear,01/01 01:00:00,E,Clear,39312,378007.7433,0.00000,1.907959e+06
1,Eclear,01/01 02:00:00,E,Clear,39312,283505.8075,0.00000,1.474193e+06
2,Eclear,01/01 03:00:00,E,Clear,16632,283505.8075,0.00000,1.457572e+06
3,Eclear,01/01 04:00:00,E,Clear,16632,283505.8075,0.00000,1.548818e+06
4,Eclear,01/01 05:00:00,E,Clear,16632,283505.8075,0.00000,1.433108e+06
...,...,...,...,...,...,...,...,...
78835,SWlowe,12/31 20:00:00,SW,Lowe,151200,226742.1508,0.00000,2.662468e+05
78836,SWlowe,12/31 21:00:00,SW,Lowe,151200,0.0000,0.00000,0.000000e+00
78837,SWlowe,12/31 22:00:00,SW,Lowe,151200,0.0000,0.00000,0.000000e+00
78838,SWlowe,12/31 23:00:00,SW,Lowe,128520,151161.4339,96109.76581,3.179052e+05


In [13]:
all_energy_group = all_energy.groupby("Scenario")

all_energy_group.sum()

,COMFENZONE:Zone Lights Electricity Energy [J](Hourly),VAV SYS 1:Air System Fan Electricity Energy [J](Hourly),MAIN CHILLER:Chiller Electricity Energy [J](Hourly),MAIN BOILER:Boiler NaturalGas Energy [J](Hourly)
Scenario,,,,
Eclear,545866776,2.047654e+09,7.830297e+09,1.773518e+10
Eexist,545866776,2.172648e+09,8.106965e+09,2.070347e+10
Elowe,545866776,1.816409e+09,6.978698e+09,1.661581e+10
NWclear,545866776,1.889273e+09,7.872693e+09,1.413944e+10
NWexist,545866776,2.092371e+09,8.424903e+09,1.728601e+10
NWlowe,545866776,1.634981e+09,6.899258e+09,1.293096e+10
SWclear,545866776,2.094524e+09,7.457810e+09,2.022509e+10
SWexist,545866776,2.272504e+09,8.052676e+09,2.347628e+10
SWlowe,545866776,1.777595e+09,6.478326e+09,1.860509e+10


In [14]:
# Export grouped file to csv
# all_energy.to_csv("Scraper_Output/msp2_all.csv", header=True, index=True)

In [15]:
# # Export grouped file to csv
# all_energy_group.to_csv("Scraper_Output/msp2_group.csv", header=True, index=True)

-----

## Get the Wanted Data (columns)

In [16]:
# Rename Columns
all_energy2 = all_energy.rename(columns={"Scenario":"Scenario",
                              "Date/Time":"Date_Time",
                             "VAV SYS 1:Air System Fan Electricity Energy [J](Hourly) ":"FanEnergy[J](Hourly)",
                             "MAIN CHILLER:Chiller Electricity Energy [J](Hourly)":"CoolingEnergy[J](Hourly)",
                            "MAIN BOILER:Boiler NaturalGas Energy [J](Hourly)":"HeatingEnergy[J](Hourly)",
                            "COMFENZONE:Zone Lights Electricity Energy [J](Hourly)":"LightingEnergy[J](Hourly)"
                             })

# all_energy2

In [17]:
list(all_energy2.columns.values)

['Scenario',
 'Date_Time',
 'Elevation',
 'GlazingSystem',
 'LightingEnergy[J](Hourly)',
 'FanEnergy[J](Hourly)',
 'CoolingEnergy[J](Hourly)',
 'HeatingEnergy[J](Hourly)']

-----

## Get Annual Energy Data
Convert units from Joules to kBtu<br>

In [18]:
# Copy dataframe to modify and leave original df intact
get_annual = all_energy2.copy(deep=True)

get_annual

,Scenario,Date_Time,Elevation,GlazingSystem,LightingEnergy[J](Hourly),FanEnergy[J](Hourly),CoolingEnergy[J](Hourly),HeatingEnergy[J](Hourly)
0,Eclear,01/01 01:00:00,E,Clear,39312,378007.7433,0.00000,1.907959e+06
1,Eclear,01/01 02:00:00,E,Clear,39312,283505.8075,0.00000,1.474193e+06
2,Eclear,01/01 03:00:00,E,Clear,16632,283505.8075,0.00000,1.457572e+06
3,Eclear,01/01 04:00:00,E,Clear,16632,283505.8075,0.00000,1.548818e+06
4,Eclear,01/01 05:00:00,E,Clear,16632,283505.8075,0.00000,1.433108e+06
...,...,...,...,...,...,...,...,...
78835,SWlowe,12/31 20:00:00,SW,Lowe,151200,226742.1508,0.00000,2.662468e+05
78836,SWlowe,12/31 21:00:00,SW,Lowe,151200,0.0000,0.00000,0.000000e+00
78837,SWlowe,12/31 22:00:00,SW,Lowe,151200,0.0000,0.00000,0.000000e+00
78838,SWlowe,12/31 23:00:00,SW,Lowe,128520,151161.4339,96109.76581,3.179052e+05


In [19]:
# list(get_annual.columns.values)

In [20]:
# Convert Joules to KBtu in dataframe
get_annual["FanEnergy(kBtu)"] = get_annual["FanEnergy[J](Hourly)"] * .00000094781712
get_annual["HeatingEnergy(kBtu)"] = get_annual["HeatingEnergy[J](Hourly)"] * .00000094781712
get_annual["CoolingEnergy(kBtu)"] = get_annual["CoolingEnergy[J](Hourly)"] * .00000094781712

# Reduce lighting energy, assuming LEDs are being used
get_annual["LightingEnergy(kBtu)"] = (get_annual["LightingEnergy[J](Hourly)"] * .00000094781712) * lighting_redux

# get_annual

In [21]:
# Drop columns
get_annual.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)",  "CoolingEnergy[J](Hourly)", "LightingEnergy[J](Hourly)"], axis=1, inplace=True)

get_annual

,Scenario,Date_Time,Elevation,GlazingSystem,FanEnergy(kBtu),HeatingEnergy(kBtu),CoolingEnergy(kBtu),LightingEnergy(kBtu)
0,Eclear,01/01 01:00:00,E,Clear,0.358282,1.808396,0.000000,0.037261
1,Eclear,01/01 02:00:00,E,Clear,0.268712,1.397265,0.000000,0.037261
2,Eclear,01/01 03:00:00,E,Clear,0.268712,1.381512,0.000000,0.015764
3,Eclear,01/01 04:00:00,E,Clear,0.268712,1.467996,0.000000,0.015764
4,Eclear,01/01 05:00:00,E,Clear,0.268712,1.358324,0.000000,0.015764
...,...,...,...,...,...,...,...,...
78835,SWlowe,12/31 20:00:00,SW,Lowe,0.214910,0.252353,0.000000,0.143310
78836,SWlowe,12/31 21:00:00,SW,Lowe,0.000000,0.000000,0.000000,0.143310
78837,SWlowe,12/31 22:00:00,SW,Lowe,0.000000,0.000000,0.000000,0.143310
78838,SWlowe,12/31 23:00:00,SW,Lowe,0.143273,0.301316,0.091094,0.121813


In [25]:
# totalsx = get_annual.groupby(["Scenario"], as_index=True)

# totalsx.sum()


In [26]:
# Add energy totals
totalEnergy = get_annual["FanEnergy(kBtu)"] + get_annual["HeatingEnergy(kBtu)"] + get_annual["LightingEnergy(kBtu)"] + get_annual["CoolingEnergy(kBtu)"]
        
get_annual["AnnualEnergy(kBtu)"] = totalEnergy

get_annual

,Scenario,Date_Time,Elevation,GlazingSystem,FanEnergy(kBtu),HeatingEnergy(kBtu),CoolingEnergy(kBtu),LightingEnergy(kBtu),AnnualEnergy(kBtu)
0,Eclear,01/01 01:00:00,E,Clear,0.358282,1.808396,0.000000,0.037261,2.203939
1,Eclear,01/01 02:00:00,E,Clear,0.268712,1.397265,0.000000,0.037261,1.703237
2,Eclear,01/01 03:00:00,E,Clear,0.268712,1.381512,0.000000,0.015764,1.665987
3,Eclear,01/01 04:00:00,E,Clear,0.268712,1.467996,0.000000,0.015764,1.752472
4,Eclear,01/01 05:00:00,E,Clear,0.268712,1.358324,0.000000,0.015764,1.642800
...,...,...,...,...,...,...,...,...,...
78835,SWlowe,12/31 20:00:00,SW,Lowe,0.214910,0.252353,0.000000,0.143310,0.610573
78836,SWlowe,12/31 21:00:00,SW,Lowe,0.000000,0.000000,0.000000,0.143310,0.143310
78837,SWlowe,12/31 22:00:00,SW,Lowe,0.000000,0.000000,0.000000,0.143310,0.143310
78838,SWlowe,12/31 23:00:00,SW,Lowe,0.143273,0.301316,0.091094,0.121813,0.657497


-----

## Get Energy per sf 
Add EUI (site) column<br>


In [27]:
# Get data per sf (energy/sf)
get_annual["FanEnergy(kBtu/sf)"] = (get_annual["FanEnergy(kBtu)"]) / sf
get_annual["HeatingEnergy(kBtu/sf)"] = (get_annual["HeatingEnergy(kBtu)"]) / sf
get_annual["CoolingEnergy(kBtu/sf)"] = (get_annual["CoolingEnergy(kBtu)"]) / sf
get_annual["LightingEnergy(kBtu/sf)"] = (get_annual["LightingEnergy(kBtu)"]) / sf

In [28]:
# Establish EUI (energy/sf)
get_annual["EUI(kBtu/sf)"] = (get_annual["FanEnergy(kBtu)"] +
                                    get_annual["HeatingEnergy(kBtu)"] +
                                    get_annual["LightingEnergy(kBtu)"] +  
                                    get_annual["CoolingEnergy(kBtu)"]) / sf
                                                       
# get_annual

-----

## Get Carbon per sf 
Add carbon/sf column

In [29]:
# Do carbon calculation using variable (lb/sf)
get_annual["CO2_Electric(lb/sf)"] = ((get_annual["FanEnergy(kBtu)"] +
                                    get_annual["LightingEnergy(kBtu)"] +
                                    get_annual["CoolingEnergy(kBtu)"]) * elec_co2) / sf
                                                       
# get_annual_ComElec

In [30]:
get_annual["CO2_Gas(lb/sf)"] = ((get_annual["HeatingEnergy(kBtu)"]) * gas_co2) / sf
                                                       
# get_annual

## Get Annual Cost Data
Convert units from Joules to kWh and therms to use cost multiplier<br>
Add cost multiplier, variables defined above

In [31]:
# Get annual costs
get_annual_costs = all_energy2.copy(deep=True)

In [32]:
# Convert Joules to KBtu in dataframe and add cost multiplier
# Gas joules => Mbtu => therms: (kbtu/1000) * 10 NOT USED FOR ELECTRIC GAS
# Elec joules => Kwh (kbtu * .000000277777778)
get_annual_costs["FanCost($)"] = (get_annual_costs["FanEnergy[J](Hourly)"] * .000000277777778) * comelec_cost
get_annual_costs["HeatingCost($)"] = ((get_annual_costs["HeatingEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * comgas_cost
get_annual_costs["CoolingCost($)"] = (get_annual_costs["CoolingEnergy[J](Hourly)"] * .000000277777778) * comelec_cost

# Reduce lighting energy, assuming LEDs are being used
get_annual_costs["LightingCost($)"] = (get_annual_costs["LightingEnergy[J](Hourly)"] * .000000277777778) * lighting_redux * comelec_cost

#get_annual_costs

In [33]:
# Drop columns
get_annual_costs.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly)", "LightingEnergy[J](Hourly)"], axis=1, inplace=True)

#get_annual_costs

In [34]:
# Add cost totals
totalCost = get_annual_costs["FanCost($)"] +  get_annual_costs["HeatingCost($)"]  + get_annual_costs["CoolingCost($)"] + get_annual_costs["LightingCost($)"]
        
get_annual_costs["AnnualCost($)"] = totalCost

get_annual_costs

,Scenario,Date_Time,Elevation,GlazingSystem,FanCost($),HeatingCost($),CoolingCost($),LightingCost($),AnnualCost($)
0,Eclear,01/01 01:00:00,E,Clear,0.013361,0.011378,0.000000,0.001390,0.026129
1,Eclear,01/01 02:00:00,E,Clear,0.010021,0.008791,0.000000,0.001390,0.020202
2,Eclear,01/01 03:00:00,E,Clear,0.010021,0.008692,0.000000,0.000588,0.019301
3,Eclear,01/01 04:00:00,E,Clear,0.010021,0.009236,0.000000,0.000588,0.019845
4,Eclear,01/01 05:00:00,E,Clear,0.010021,0.008546,0.000000,0.000588,0.019155
...,...,...,...,...,...,...,...,...,...
78835,SWlowe,12/31 20:00:00,SW,Lowe,0.008015,0.001588,0.000000,0.005344,0.014947
78836,SWlowe,12/31 21:00:00,SW,Lowe,0.000000,0.000000,0.000000,0.005344,0.005344
78837,SWlowe,12/31 22:00:00,SW,Lowe,0.000000,0.000000,0.000000,0.005344,0.005344
78838,SWlowe,12/31 23:00:00,SW,Lowe,0.005343,0.001896,0.003397,0.004543,0.015179


In [1]:
# Need to add square footage per elevation to the export, or add it via Tableau (if equal on all elevations)
costx = get_annual_costs.groupby(["Scenario"], as_index=True)

costx.sum()

NameError: name 'get_annual_costs' is not defined

-----

## Get Annual Cost per sf Data
Add costs/sf columns<br>


In [36]:
# Get data per sf ($/sf)
get_annual_costs["FanCost($/sf)"] = (get_annual_costs["FanCost($)"]) / sf
get_annual_costs["HeatingCost($/sf)"] = (get_annual_costs["HeatingCost($)"]) / sf
get_annual_costs["CoolingCost($/sf)"] = (get_annual_costs["CoolingCost($)"]) / sf
get_annual_costs["LightingCost($/sf)"] = (get_annual_costs["LightingCost($)"]) / sf

# get_annual_costs_ComElec

In [37]:
# Establish costs ($/sf)
get_annual_costs["AnnualCost($/sf)"] = (get_annual_costs["FanCost($)"] +
                                    get_annual_costs["HeatingCost($)"] +  
                                    get_annual_costs["CoolingCost($)"] +  
                                    get_annual_costs["LightingCost($)"]) / sf
                                                       
# get_annual_costs

---

## Combine Annual Energy and Cost Data
Combine two dfs<br>
Export combine file

In [38]:
# # Combine data for energy and cost
allAnnualData = pd.merge(get_annual, get_annual_costs)

# allAnnualData

In [39]:
# list(allAnnualDataComElec.columns.values)

---
## Group and Export Annual Data
Group dataframe by scenario<br>
Export to file

In [40]:
# Group by scenario
annualData = allAnnualData.groupby(["Scenario", "Elevation", "GlazingSystem"], as_index=True)

# annualData.sum()

In [41]:
annualDataFinal = annualData.sum()

# annualDataFinal

In [42]:
# list(annualDataFinal.columns.values)

In [43]:
# Export grouped file to csv
annualDataFinal.to_csv("Scraper_Output/msp2_annual_com.csv", header=True, index=True)

---

## Establish dataframes for each condition to determine delta from existing

Create dataset with just existing condition.<br> 
Created a dataset with clear panel. <br>
Create a dataset with lowe panel.<br>
Keep the units in kBtus to be able to eventually combine heating and cooling <br>
Export to CSV files


In [44]:
# Define the scenario for the existing condition.
baselineAnnualData = allAnnualData[allAnnualData["GlazingSystem"] == "Exist"]

# baselineAnnualData

In [45]:
# Rename columns to indicate baseline to prepare for merge"

allBaselineAnnualData = baselineAnnualData.rename(columns={'Scenario':'B_Scenario',
 'Date_Time':'B_Date_Time',
 'Elevation':'B_Elevation',
 'GlazingSystem':'B_GlazingSystem',
 'FanEnergy(kBtu)':'B_FanEnergy(kBtu)',
 'HeatingEnergy(kBtu)':'B_HeatingEnergy(kBtu)',
 'CoolingEnergy(kBtu)':'B_CoolingEnergy(kBtu)',
 'LightingEnergy(kBtu)':'B_LightingEnergy(kBtu)',
 'AnnualEnergy(kBtu)':'B_AnnualEnergy(kBtu)',
 'FanEnergy(kBtu/sf)':'B_FanEnergy(kBtu/sf)',
 'HeatingEnergy(kBtu/sf)':'B_HeatingEnergy(kBtu/sf)',
 'CoolingEnergy(kBtu/sf)':'B_CoolingEnergy(kBtu/sf)',
 'LightingEnergy(kBtu/sf)':'B_LightingEnergy(kBtu/sf)',
 'EUI(kBtu/sf)':'B_EUI(kBtu/sf)',
 'CO2_Electric(lb/sf)':'B_CO2_Electric(lb/sf)',
 'CO2_Gas(lb/sf)':'B_CO2_Gas(lb/sf)',
 'FanCost($)':'B_FanCost($)',
 'HeatingCost($)':'B_HeatingCost($)',
 'CoolingCost($)':'B_CoolingCost($)',
 'LightingCost($)':'B_LightingCost($)',
 'AnnualCost($)':'B_AnnualCost($)',
 'FanCost($/sf)':'B_FanCost($/sf)',
 'HeatingCost($/sf)':'B_HeatingCost($/sf)',
 'CoolingCost($/sf)':'B_CoolingCost($/sf)',
 'LightingCost($/sf)':'B_LightingCost($/sf)',
 'AnnualCost($/sf)':'B_AnnualCost($/sf)'
                            })

# allBaselineAnnualDataComElec

In [46]:
# Reset index
allBaselineAnnualData.reset_index(inplace=True, drop=True)

# allBaselineAnnualData

In [47]:
# # Define the scenario for the baselines.

clearAnnualData = allAnnualData[allAnnualData["GlazingSystem"] == "Clear"]

# clearAnnualData

In [48]:
# Rename columns to indicate baseline to prepare for merge"

allClearAnnualData = clearAnnualData.rename(columns={'Scenario':'C_Scenario',
 'Date_Time':'C_Date_Time',
 'Elevation':'C_Elevation',
 'GlazingSystem':'C_GlazingSystem',
 'FanEnergy(kBtu)':'C_FanEnergy(kBtu)',
 'HeatingEnergy(kBtu)':'C_HeatingEnergy(kBtu)',
 'CoolingEnergy(kBtu)':'C_CoolingEnergy(kBtu)',
 'LightingEnergy(kBtu)':'C_LightingEnergy(kBtu)',
 'AnnualEnergy(kBtu)':'C_AnnualEnergy(kBtu)',
 'FanEnergy(kBtu/sf)':'C_FanEnergy(kBtu/sf)',
 'HeatingEnergy(kBtu/sf)':'C_HeatingEnergy(kBtu/sf)',
 'CoolingEnergy(kBtu/sf)':'C_CoolingEnergy(kBtu/sf)',
 'LightingEnergy(kBtu/sf)':'C_LightingEnergy(kBtu/sf)',
 'EUI(kBtu/sf)':'C_EUI(kBtu/sf)',
 'CO2_Electric(lb/sf)':'C_CO2_Electric(lb/sf)',
 'CO2_Gas(lb/sf)':'C_CO2_Gas(lb/sf)',
 'FanCost($)':'C_FanCost($)',
 'HeatingCost($)':'C_HeatingCost($)',
 'CoolingCost($)':'C_CoolingCost($)',
 'LightingCost($)':'C_LightingCost($)',
 'AnnualCost($)':'C_AnnualCost($)',
 'FanCost($/sf)':'C_FanCost($/sf)',
 'HeatingCost($/sf)':'C_HeatingCost($/sf)',
 'CoolingCost($/sf)':'C_CoolingCost($/sf)',
 'LightingCost($/sf)':'C_LightingCost($/sf)',
 'AnnualCost($/sf)':'C_AnnualCost($/sf)'
                            })

# allClearAnnualData

In [49]:
# Reset index
allClearAnnualData.reset_index(inplace=True, drop=True)

# allClearAnnualDataComElec

In [50]:
# Define the scenario for the baselines.

loweAnnualData = allAnnualData[allAnnualData["GlazingSystem"] == "Lowe"]

# loweAnnualDataComElec

In [51]:
# Rename columns to indicate baseline to prepare for merge"

allLoweAnnualData = loweAnnualData.rename(columns={'Scenario':'L_Scenario',
 'Date_Time':'L_Date_Time',
 'Elevation':'L_Elevation',
 'GlazingSystem':'L_GlazingSystem',
 'FanEnergy(kBtu)':'L_FanEnergy(kBtu)',
 'HeatingEnergy(kBtu)':'L_HeatingEnergy(kBtu)',
 'CoolingEnergy(kBtu)':'L_CoolingEnergy(kBtu)',
 'LightingEnergy(kBtu)':'L_LightingEnergy(kBtu)',
 'AnnualEnergy(kBtu)':'L_AnnualEnergy(kBtu)',
 'FanEnergy(kBtu/sf)':'L_FanEnergy(kBtu/sf)',
 'HeatingEnergy(kBtu/sf)':'L_HeatingEnergy(kBtu/sf)',
 'CoolingEnergy(kBtu/sf)':'L_CoolingEnergy(kBtu/sf)',
 'LightingEnergy(kBtu/sf)':'L_LightingEnergy(kBtu/sf)',
 'EUI(kBtu/sf)':'L_EUI(kBtu/sf)',
 'CO2_Electric(lb/sf)':'L_CO2_Electric(lb/sf)',
 'CO2_Gas(lb/sf)':'L_CO2_Gas(lb/sf)',
 'FanCost($)':'L_FanCost($)',
 'HeatingCost($)':'L_HeatingCost($)',
 'CoolingCost($)':'L_CoolingCost($)',
 'LightingCost($)':'L_LightingCost($)',
 'AnnualCost($)':'L_AnnualCost($)',
 'FanCost($/sf)':'L_FanCost($/sf)',
 'HeatingCost($/sf)':'L_HeatingCost($/sf)',
 'CoolingCost($/sf)':'L_CoolingCost($/sf)',
 'LightingCost($/sf)':'L_LightingCost($/sf)',
 'AnnualCost($/sf)':'L_AnnualCost($/sf)'
                            })

# allLoweAnnualDataComElec

In [52]:
#Reset index
allLoweAnnualData.reset_index(inplace=True, drop=True)

# allLoweAnnualDataComElec

---
## Merge the 3 dataframes to perform delta from existing calculation


In [53]:
# merge existing and clear
MergedAnnualData = pd.merge(allBaselineAnnualData, allClearAnnualData, left_index=True, right_index=True)

# merge the above with lowe
AllMergedAnnualData = pd.merge(MergedAnnualData, allLoweAnnualData, left_index=True, right_index=True)

# AllMergedAnnualData

In [54]:
# list(AllMergedAnnualData.columns.values)

In [55]:
# Deltas of exist - clear

AllMergedAnnualData["DeltaClear_FanEnergy(kBtu/sf)"] = AllMergedAnnualData["B_FanEnergy(kBtu/sf)"] - AllMergedAnnualData["C_FanEnergy(kBtu/sf)"]
AllMergedAnnualData["DeltaClear_HeatingEnergy(kBtu/sf)"] = AllMergedAnnualData["B_HeatingEnergy(kBtu/sf)"] - AllMergedAnnualData["C_HeatingEnergy(kBtu/sf)"]
AllMergedAnnualData["DeltaClear_CoolingEnergy(kBtu/sf)"] = AllMergedAnnualData["B_CoolingEnergy(kBtu/sf)"] - AllMergedAnnualData["C_CoolingEnergy(kBtu/sf)"]
AllMergedAnnualData["DeltaClear_LightingEnergy(kBtu/sf)"] = AllMergedAnnualData["B_LightingEnergy(kBtu/sf)"] - AllMergedAnnualData["C_LightingEnergy(kBtu/sf)"]
AllMergedAnnualData["DeltaClear_EUI(kBtu/sf)"] = AllMergedAnnualData["B_EUI(kBtu/sf)"] - AllMergedAnnualData["C_EUI(kBtu/sf)"]
AllMergedAnnualData["DeltaClear_CO2_Electric(lb/sf)"] = AllMergedAnnualData["B_CO2_Electric(lb/sf)"] - AllMergedAnnualData["C_CO2_Electric(lb/sf)"]
AllMergedAnnualData["DeltaClear_CO2_Gas(lb/sf)"] = AllMergedAnnualData["B_CO2_Gas(lb/sf)"] - AllMergedAnnualData["C_CO2_Gas(lb/sf)"]
AllMergedAnnualData["DeltaClear_FanCost($)"] = AllMergedAnnualData["B_FanCost($)"] - AllMergedAnnualData["C_FanCost($)"]
AllMergedAnnualData["DeltaClear_HeatingCost($)"] = AllMergedAnnualData["B_HeatingCost($)"] - AllMergedAnnualData["C_HeatingCost($)"]
AllMergedAnnualData["DeltaClear_CoolingCost($)"] = AllMergedAnnualData["B_CoolingCost($)"] - AllMergedAnnualData["C_CoolingCost($)"]
AllMergedAnnualData["DeltaClear_LightingCost($)"] = AllMergedAnnualData["B_LightingCost($)"] - AllMergedAnnualData["C_LightingCost($)"]
AllMergedAnnualData["DeltaClear_AnnualCost($)"] = AllMergedAnnualData["B_AnnualCost($)"] - AllMergedAnnualData["C_AnnualCost($)"]
AllMergedAnnualData["DeltaClear_FanCost($/sf)"] = AllMergedAnnualData["B_FanCost($/sf)"] - AllMergedAnnualData["C_FanCost($/sf)"]
AllMergedAnnualData["DeltaClear_HeatingCost($/sf)"] = AllMergedAnnualData["B_HeatingCost($/sf)"] - AllMergedAnnualData["C_HeatingCost($/sf)"]
AllMergedAnnualData["DeltaClear_CoolingCost($/sf)"] = AllMergedAnnualData["B_CoolingCost($/sf)"] - AllMergedAnnualData["C_CoolingCost($/sf)"]
AllMergedAnnualData["DeltaClear_LightingCost($/sf)"] = AllMergedAnnualData["B_LightingCost($/sf)"] - AllMergedAnnualData["C_LightingCost($/sf)"]
AllMergedAnnualData["DeltaClear_AnnualCost($/sf)"] = AllMergedAnnualData["B_AnnualCost($/sf)"] - AllMergedAnnualData["C_AnnualCost($/sf)"]


In [56]:
# AllMergedAnnualData

In [57]:
# list(AllMergedAnnualData.columns.values)

In [58]:
# Deltas of exist - lowe

AllMergedAnnualData["DeltaLowe_FanEnergy(kBtu/sf)"] = AllMergedAnnualData["B_FanEnergy(kBtu/sf)"] - AllMergedAnnualData["L_FanEnergy(kBtu/sf)"]
AllMergedAnnualData["DeltaLowe_HeatingEnergy(kBtu/sf)"] = AllMergedAnnualData["B_HeatingEnergy(kBtu/sf)"] - AllMergedAnnualData["L_HeatingEnergy(kBtu/sf)"]
AllMergedAnnualData["DeltaLowe_CoolingEnergy(kBtu/sf)"] = AllMergedAnnualData["B_CoolingEnergy(kBtu/sf)"] - AllMergedAnnualData["L_CoolingEnergy(kBtu/sf)"]
AllMergedAnnualData["DeltaLowe_LightingEnergy(kBtu/sf)"] = AllMergedAnnualData["B_LightingEnergy(kBtu/sf)"] - AllMergedAnnualData["L_LightingEnergy(kBtu/sf)"]
AllMergedAnnualData["DeltaLowe_EUI(kBtu/sf)"] = AllMergedAnnualData["B_EUI(kBtu/sf)"] - AllMergedAnnualData["L_EUI(kBtu/sf)"]
AllMergedAnnualData["DeltaLowe_CO2_Electric(lb/sf)"] = AllMergedAnnualData["B_CO2_Electric(lb/sf)"] - AllMergedAnnualData["L_CO2_Electric(lb/sf)"]
AllMergedAnnualData["DeltaLowe_CO2_Gas(lb/sf)"] = AllMergedAnnualData["B_CO2_Gas(lb/sf)"] - AllMergedAnnualData["L_CO2_Gas(lb/sf)"]
AllMergedAnnualData["DeltaLowe_FanCost($)"] = AllMergedAnnualData["B_FanCost($)"] - AllMergedAnnualData["L_FanCost($)"]
AllMergedAnnualData["DeltaLowe_HeatingCost($)"] = AllMergedAnnualData["B_HeatingCost($)"] - AllMergedAnnualData["L_HeatingCost($)"]
AllMergedAnnualData["DeltaLowe_CoolingCost($)"] = AllMergedAnnualData["B_CoolingCost($)"] - AllMergedAnnualData["L_CoolingCost($)"]
AllMergedAnnualData["DeltaLowe_LightingCost($)"] = AllMergedAnnualData["B_LightingCost($)"] - AllMergedAnnualData["L_LightingCost($)"]
AllMergedAnnualData["DeltaLowe_AnnualCost($)"] = AllMergedAnnualData["B_AnnualCost($)"] - AllMergedAnnualData["L_AnnualCost($)"]
AllMergedAnnualData["DeltaLowe_FanCost($/sf)"] = AllMergedAnnualData["B_FanCost($/sf)"] - AllMergedAnnualData["L_FanCost($/sf)"]
AllMergedAnnualData["DeltaLowe_HeatingCost($/sf)"] = AllMergedAnnualData["B_HeatingCost($/sf)"] - AllMergedAnnualData["L_HeatingCost($/sf)"]
AllMergedAnnualData["DeltaLowe_CoolingCost($/sf)"] = AllMergedAnnualData["B_CoolingCost($/sf)"] - AllMergedAnnualData["L_CoolingCost($/sf)"]
AllMergedAnnualData["DeltaLowe_LightingCost($/sf)"] = AllMergedAnnualData["B_LightingCost($/sf)"] - AllMergedAnnualData["L_LightingCost($/sf)"]
AllMergedAnnualData["DeltaLowe_AnnualCost($/sf)"] = AllMergedAnnualData["B_AnnualCost($/sf)"] - AllMergedAnnualData["L_AnnualCost($/sf)"]


In [59]:
# list(AllMergedAnnualData.columns.values)

In [60]:
# AllMergedAnnualData

---
## Group and Export Annual Delta Data
Group dataframe by scenario<br>
Export to file

In [61]:
# Group by scenario
AnnualDataDelta = AllMergedAnnualData.groupby(["B_Scenario", "B_Elevation", "B_GlazingSystem"], as_index=False)

# AnnualDataDelta.sum()

In [62]:
# list(AnnualDataDelta.sum().columns.values)

In [63]:
AnnualDataDelta = AnnualDataDelta.sum()

In [64]:
# Export grouped file to csv
AnnualDataDelta.to_csv("Scraper_Output/msp2_deltas_annual_com.csv", header=True, index=False)

## Find Peak Demand
Slice data to find peak demand for each scenario<br>
Export to csv

In [65]:
# Copy dataframe to modify and leave original df intact
get_peak = all_energy2.copy(deep=True)

# get_peak

In [66]:
# list(get_peak.columns.values)

In [67]:
# Add energy totals
# divide by the number of seconds in an hour (3600)
totalPeakElec = (get_peak["FanEnergy[J](Hourly)"]  + get_peak["CoolingEnergy[J](Hourly)"] + get_peak["LightingEnergy[J](Hourly)"])
totalPeakGas =  get_peak["HeatingEnergy[J](Hourly)"]
        
get_peak["PeakElecEnergy[J](Hourly)"] = totalPeakElec
get_peak["PeakGasEnergy[J](Hourly)"] = totalPeakGas


get_peak["PeakElecEnergy(W)"] = (get_peak["PeakElecEnergy[J](Hourly)"] / 3600)
get_peak["PeakElecEnergy(W/sf)"] = (get_peak["PeakElecEnergy(W)"] / sf)


get_peak["PeakGasEnergy(W)"] = (get_peak["PeakGasEnergy[J](Hourly)"] / 3600)
get_peak["PeakGasEnergy(W/sf)"] = (get_peak["PeakGasEnergy(W)"] / sf)

# get_peak

In [68]:
# get_peak["PeakGasEnergy[J](Hourly)"].idxmax()

In [69]:
# get_peak["PeakElecEnergy[J](Hourly)"].idxmax()

In [70]:
# Get scenario
Eexist = get_peak[get_peak["Scenario"] == "Eexist"]
Eclear = get_peak[get_peak["Scenario"] == "Eclear"]
Elowe = get_peak[get_peak["Scenario"] == "Elowe"]
NWexist = get_peak[get_peak["Scenario"] == "NWexist"]
NWclear = get_peak[get_peak["Scenario"] == "NWclear"]
NWlowe = get_peak[get_peak["Scenario"] == "NWlowe"]
SWexist = get_peak[get_peak["Scenario"] == "SWexist"]
SWclear = get_peak[get_peak["Scenario"] == "SWclear"]
SWlowe = get_peak[get_peak["Scenario"] == "SWlowe"]


In [71]:
EexistPeakGas = pd.DataFrame(Eexist.loc[Eexist['HeatingEnergy[J](Hourly)'].idxmax()])
EexistPeakGas = EexistPeakGas.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
EclearPeaKGas = pd.DataFrame(Eclear.loc[Eclear['HeatingEnergy[J](Hourly)'].idxmax()])
EclearPeaKGas = EclearPeaKGas.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
ElowePeakGas = pd.DataFrame(Elowe.loc[Elowe['HeatingEnergy[J](Hourly)'].idxmax()])
ElowePeakGas = ElowePeakGas.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
NWexistPeakGas = pd.DataFrame(NWexist.loc[NWexist['HeatingEnergy[J](Hourly)'].idxmax()])
NWexistPeakGas = NWexistPeakGas.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
NWclearPeakGas = pd.DataFrame(NWclear.loc[NWclear['HeatingEnergy[J](Hourly)'].idxmax()])
NWclearPeakGas = NWclearPeakGas.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
NWlowePeakGas = pd.DataFrame(NWlowe.loc[NWlowe['HeatingEnergy[J](Hourly)'].idxmax()])
NWlowePeakGas = NWlowePeakGas.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
SWexistPeakGas = pd.DataFrame(SWexist.loc[SWexist['HeatingEnergy[J](Hourly)'].idxmax()])
SWexistPeakGas = SWexistPeakGas.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
SWclearPeakGas = pd.DataFrame(SWclear.loc[SWclear['HeatingEnergy[J](Hourly)'].idxmax()])
SWclearPeakGas =  SWclearPeakGas.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
SWlowePeakGas = pd.DataFrame(SWlowe.loc[SWlowe['HeatingEnergy[J](Hourly)'].idxmax()])
SWlowePeakGas = SWlowePeakGas.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)

# EexistPeakGas

In [72]:
EexistPeakElec = pd.DataFrame(Eexist.loc[Eexist['CoolingEnergy[J](Hourly)'].idxmax()])
EexistPeakElec = EexistPeakElec.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
EclearPeaKElec = pd.DataFrame(Eclear.loc[Eclear['CoolingEnergy[J](Hourly)'].idxmax()])
EclearPeaKElec = EclearPeaKElec.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
ElowePeakElec = pd.DataFrame(Elowe.loc[Elowe['CoolingEnergy[J](Hourly)'].idxmax()])
ElowePeakElec = ElowePeakElec.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
NWexistPeakElec = pd.DataFrame(NWexist.loc[NWexist['CoolingEnergy[J](Hourly)'].idxmax()])
NWexistPeakElec = NWexistPeakElec.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
NWclearPeakElec = pd.DataFrame(NWclear.loc[NWclear['CoolingEnergy[J](Hourly)'].idxmax()])
NWclearPeakElec = NWclearPeakElec.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
NWlowePeakElec = pd.DataFrame(NWlowe.loc[NWlowe['CoolingEnergy[J](Hourly)'].idxmax()])
NWlowePeakElec = NWlowePeakElec.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
SWexistPeakElec = pd.DataFrame(SWexist.loc[SWexist['CoolingEnergy[J](Hourly)'].idxmax()])
SWexistPeakElec = SWexistPeakElec.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
SWclearPeakElec = pd.DataFrame(SWclear.loc[SWclear['CoolingEnergy[J](Hourly)'].idxmax()])
SWclearPeakElec =  SWclearPeakElec.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
SWlowePeakElec = pd.DataFrame(SWlowe.loc[SWlowe['CoolingEnergy[J](Hourly)'].idxmax()])
SWlowePeakElec = SWlowePeakElec.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)

# EexistPeakElec

In [73]:
PeakDemandElec = EexistPeakElec.append([EclearPeaKElec,
ElowePeakElec,
NWexistPeakElec,
NWclearPeakElec,
NWlowePeakElec,
SWexistPeakElec,
SWclearPeakElec,
SWlowePeakElec
])

PeakDemandElec

,Scenario,Date_Time,Elevation,GlazingSystem,LightingEnergy[J](Hourly),FanEnergy[J](Hourly),CoolingEnergy[J](Hourly),HeatingEnergy[J](Hourly),PeakElecEnergy[J](Hourly),PeakGasEnergy[J](Hourly),PeakElecEnergy(W),PeakElecEnergy(W/sf),PeakGasEnergy(W),PeakGasEnergy(W/sf)
0,Eexist,08/02 09:00:00,E,Exist,66528,670965,5.23197e+06,0,5.96946e+06,0,1658.18,7.89612,0,0
0,Eclear,08/02 09:00:00,E,Clear,66528,664015,5.01435e+06,0,5.7449e+06,0,1595.8,7.59907,0,0
0,Elowe,08/02 09:00:00,E,Lowe,66528,629660,4.49436e+06,0,5.19055e+06,0,1441.82,6.8658,0,0
0,NWexist,07/05 17:00:00,NW,Exist,42336,657737,4.98142e+06,0,5.68149e+06,0,1578.19,7.5152,0,0
0,NWclear,07/24 08:00:00,NW,Clear,84672,461532,4.67848e+06,0,5.22469e+06,0,1451.3,6.91096,0,0
0,NWlowe,07/06 08:00:00,NW,Lowe,84672,428876,4.29955e+06,0,4.8131e+06,0,1336.97,6.36653,0,0
0,SWexist,07/05 19:00:00,SW,Exist,101304,735715,5.06429e+06,0,5.90131e+06,0,1639.25,7.80596,0,0
0,SWclear,07/05 18:00:00,SW,Clear,42336,636015,4.66139e+06,0,5.33974e+06,0,1483.26,7.06315,0,0
0,SWlowe,07/05 18:00:00,SW,Lowe,42336,557972,4.19014e+06,0,4.79045e+06,0,1330.68,6.33658,0,0


In [74]:
PeakDemandGas = EexistPeakGas.append([EclearPeaKGas,
ElowePeakGas,
NWexistPeakGas,
NWclearPeakGas,
NWlowePeakGas,
SWexistPeakGas,
SWclearPeakGas,
SWlowePeakGas
])

PeakDemandGas

,Scenario,Date_Time,Elevation,GlazingSystem,LightingEnergy[J](Hourly),FanEnergy[J](Hourly),CoolingEnergy[J](Hourly),HeatingEnergy[J](Hourly),PeakElecEnergy[J](Hourly),PeakGasEnergy[J](Hourly),PeakElecEnergy(W),PeakElecEnergy(W/sf),PeakGasEnergy(W),PeakGasEnergy(W/sf)
0,Eexist,01/03 09:00:00,E,Exist,66528,944908,2.59717e+06,1.5253e+07,3.60861e+06,1.5253e+07,1002.39,4.77329,4236.95,20.176
0,Eclear,01/03 08:00:00,E,Clear,84672,894950,2.37477e+06,1.26073e+07,3.35439e+06,1.26073e+07,931.776,4.43703,3502.03,16.6763
0,Elowe,01/03 09:00:00,E,Lowe,66528,797576,2.01047e+06,1.19041e+07,2.87457e+06,1.19041e+07,798.493,3.80235,3306.68,15.7461
0,NWexist,01/03 09:00:00,NW,Exist,66528,934438,2.5026e+06,1.44454e+07,3.50357e+06,1.44454e+07,973.213,4.63435,4012.61,19.1077
0,NWclear,01/03 08:00:00,NW,Clear,84672,852679,2.16256e+06,1.17628e+07,3.09991e+06,1.17628e+07,861.087,4.10041,3267.44,15.5592
0,NWlowe,01/03 08:00:00,NW,Lowe,84672,755277,1.89757e+06,1.10567e+07,2.73752e+06,1.10567e+07,760.421,3.62105,3071.29,14.6252
0,SWexist,01/03 08:00:00,SW,Exist,84672,899766,2.48266e+06,1.41844e+07,3.4671e+06,1.41844e+07,963.083,4.58611,3940.11,18.7624
0,SWclear,01/03 08:00:00,SW,Clear,84672,818592,2.04476e+06,1.15062e+07,2.94802e+06,1.15062e+07,818.895,3.8995,3196.18,15.2199
0,SWlowe,01/03 09:00:00,SW,Lowe,66528,715763,1.73638e+06,1.11343e+07,2.51867e+06,1.11343e+07,699.63,3.33157,3092.85,14.7279


In [75]:
# # Export dataframe  to csv
# PeakDemandElec.to_csv("Scraper_Output/msp2_peakelec_com.csv", header=True, index=False)
# PeakDemandGas.to_csv("Scraper_Output/msp2_peakgas_com.csv", header=True, index=False)

---

## Establish dataframes for each condition to determine delta from existing

Create dataset with just existing condition.<br> 
Created a dataset with clear panel. <br>
Create a dataset with lowe panel.<br>
Export to CSV files


In [76]:
# Define the scenario for the existing condition.
baselinePeakDemandElec = PeakDemandElec[PeakDemandElec["GlazingSystem"] == "Exist"]

# baselinePeakDemandElec

In [77]:
# list(baselinePeakDemandElec.columns.values)

In [78]:
# Rename columns to indicate baseline to prepare for merge"

allbaselinePeakDemandElec = baselinePeakDemandElec.rename(columns={'Scenario':'B_Scenario',
 'Date_Time':'B_Date_Time',
 'Elevation':'B_Elevation',
 'GlazingSystem':'B_GlazingSystem',
 'HeatingEnergy[J](Hourly)':'B_HeatingEnergy[J](Hourly)',
 'FanEnergy[J](Hourly)':'B_FanEnergy[J](Hourly)',
 'CoolingEnergy[J](Hourly)':'B_CoolingEnergy[J](Hourly)',
 'LightingEnergy[J](Hourly)':'B_LightingEnergy[J](Hourly)',
 'PeakElecEnergy[J](Hourly)':'B_PeakElecEnergy[J](Hourly)',
 'PeakGasEnergy[J](Hourly)':'B_PeakGasEnergy[J](Hourly)',
 'PeakElecEnergy(W)':'B_PeakElecEnergy(W)',
 'PeakElecEnergy(W/sf)':'B_PeakElecEnergy(W/sf)',
'PeakGasEnergy(W)':'B_PeakGasEnergy(W)',
 'PeakGasEnergy(W/sf)':'B_PeakGasEnergy(W/sf)'

                            })

allbaselinePeakDemandElec

,B_Scenario,B_Date_Time,B_Elevation,B_GlazingSystem,B_LightingEnergy[J](Hourly),B_FanEnergy[J](Hourly),B_CoolingEnergy[J](Hourly),B_HeatingEnergy[J](Hourly),B_PeakElecEnergy[J](Hourly),B_PeakGasEnergy[J](Hourly),B_PeakElecEnergy(W),B_PeakElecEnergy(W/sf),B_PeakGasEnergy(W),B_PeakGasEnergy(W/sf)
0,Eexist,08/02 09:00:00,E,Exist,66528,670965,5.23197e+06,0,5.96946e+06,0,1658.18,7.89612,0,0
0,NWexist,07/05 17:00:00,NW,Exist,42336,657737,4.98142e+06,0,5.68149e+06,0,1578.19,7.5152,0,0
0,SWexist,07/05 19:00:00,SW,Exist,101304,735715,5.06429e+06,0,5.90131e+06,0,1639.25,7.80596,0,0


In [79]:
#Reset index
allbaselinePeakDemandElec.reset_index(inplace=True, drop=True)

allbaselinePeakDemandElec

,B_Scenario,B_Date_Time,B_Elevation,B_GlazingSystem,B_LightingEnergy[J](Hourly),B_FanEnergy[J](Hourly),B_CoolingEnergy[J](Hourly),B_HeatingEnergy[J](Hourly),B_PeakElecEnergy[J](Hourly),B_PeakGasEnergy[J](Hourly),B_PeakElecEnergy(W),B_PeakElecEnergy(W/sf),B_PeakGasEnergy(W),B_PeakGasEnergy(W/sf)
0,Eexist,08/02 09:00:00,E,Exist,66528,670965,5.23197e+06,0,5.96946e+06,0,1658.18,7.89612,0,0
1,NWexist,07/05 17:00:00,NW,Exist,42336,657737,4.98142e+06,0,5.68149e+06,0,1578.19,7.5152,0,0
2,SWexist,07/05 19:00:00,SW,Exist,101304,735715,5.06429e+06,0,5.90131e+06,0,1639.25,7.80596,0,0


In [80]:
# Define the scenario for the clear panel condition.
clearPeakDemandElec = PeakDemandElec[PeakDemandElec["GlazingSystem"] == "Clear"]

clearPeakDemandElec

,Scenario,Date_Time,Elevation,GlazingSystem,LightingEnergy[J](Hourly),FanEnergy[J](Hourly),CoolingEnergy[J](Hourly),HeatingEnergy[J](Hourly),PeakElecEnergy[J](Hourly),PeakGasEnergy[J](Hourly),PeakElecEnergy(W),PeakElecEnergy(W/sf),PeakGasEnergy(W),PeakGasEnergy(W/sf)
0,Eclear,08/02 09:00:00,E,Clear,66528,664015,5.01435e+06,0,5.7449e+06,0,1595.8,7.59907,0,0
0,NWclear,07/24 08:00:00,NW,Clear,84672,461532,4.67848e+06,0,5.22469e+06,0,1451.3,6.91096,0,0
0,SWclear,07/05 18:00:00,SW,Clear,42336,636015,4.66139e+06,0,5.33974e+06,0,1483.26,7.06315,0,0


In [81]:
# Rename columns to indicate baseline to prepare for merge"

allClearPeakDemandElec = clearPeakDemandElec.rename(columns={'Scenario':'C_Scenario',
 'Date_Time':'C_Date_Time',
 'Elevation':'C_Elevation',
 'GlazingSystem':'C_GlazingSystem',
 'HeatingEnergy[J](Hourly)':'C_HeatingEnergy[J](Hourly)',
 'FanEnergy[J](Hourly)':'C_FanEnergy[J](Hourly)',
 'CoolingEnergy[J](Hourly)':'C_CoolingEnergy[J](Hourly)',
 'LightingEnergy[J](Hourly)':'C_LightingEnergy[J](Hourly)',
 'PeakElecEnergy[J](Hourly)':'C_PeakElecEnergy[J](Hourly)',
  'PeakGasEnergy[J](Hourly)':'C_PeakGasEnergy[J](Hourly)',
'PeakElecEnergy(W)':'C_PeakElecEnergy(W)',
 'PeakElecEnergy(W/sf)':'C_PeakElecEnergy(W/sf)',
'PeakGasEnergy(W)':'C_PeakGasEnergy(W)',
 'PeakGasEnergy(W/sf)':'C_PeakGasEnergy(W/sf)'
})

# allClearPeakDemandElec

In [82]:
#Reset index
allClearPeakDemandElec.reset_index(inplace=True, drop=True)

# allClearPeakDemandElec

In [83]:
# Define the scenario for the lowe panel condition.
lowePeakDemandElec = PeakDemandElec[PeakDemandElec["GlazingSystem"] == "Lowe"]

lowePeakDemandElec

,Scenario,Date_Time,Elevation,GlazingSystem,LightingEnergy[J](Hourly),FanEnergy[J](Hourly),CoolingEnergy[J](Hourly),HeatingEnergy[J](Hourly),PeakElecEnergy[J](Hourly),PeakGasEnergy[J](Hourly),PeakElecEnergy(W),PeakElecEnergy(W/sf),PeakGasEnergy(W),PeakGasEnergy(W/sf)
0,Elowe,08/02 09:00:00,E,Lowe,66528,629660,4.49436e+06,0,5.19055e+06,0,1441.82,6.8658,0,0
0,NWlowe,07/06 08:00:00,NW,Lowe,84672,428876,4.29955e+06,0,4.8131e+06,0,1336.97,6.36653,0,0
0,SWlowe,07/05 18:00:00,SW,Lowe,42336,557972,4.19014e+06,0,4.79045e+06,0,1330.68,6.33658,0,0


In [84]:
# Rename columns to indicate baseline to prepare for merge"

allLowePeakDemandElec = lowePeakDemandElec.rename(columns={'Scenario':'L_Scenario',
 'Date_Time':'L_Date_Time',
 'Elevation':'L_Elevation',
 'GlazingSystem':'L_GlazingSystem',
 'HeatingEnergy[J](Hourly)':'L_HeatingGasEnergy[J](Hourly)',
 'FanEnergy[J](Hourly)':'L_FanEnergy[J](Hourly)',
 'CoolingEnergy[J](Hourly)':'L_CoolingEnergy[J](Hourly)',
 'LightingEnergy[J](Hourly)':'L_LightingEnergy[J](Hourly)',
 'PeakElecEnergy[J](Hourly)':'L_PeakElecEnergy[J](Hourly)',
 'PeakGasEnergy[J](Hourly)':'L_PeakGasEnergy[J](Hourly)',
 'PeakElecEnergy(W)':'L_PeakElecEnergy(W)',
 'PeakElecEnergy(W/sf)':'L_PeakElecEnergy(W/sf)',
 'PeakGasEnergy(W)':'L_PeakGasEnergy(W)',
 'PeakGasEnergy(W/sf)':'L_PeakGasEnergy(W/sf)'
                            })

# allLowePeakDemandElec

In [85]:
# Reset index
allLowePeakDemandElec.reset_index(inplace=True, drop=True)

allLowePeakDemandElec

,L_Scenario,L_Date_Time,L_Elevation,L_GlazingSystem,L_LightingEnergy[J](Hourly),L_FanEnergy[J](Hourly),L_CoolingEnergy[J](Hourly),L_HeatingGasEnergy[J](Hourly),L_PeakElecEnergy[J](Hourly),L_PeakGasEnergy[J](Hourly),L_PeakElecEnergy(W),L_PeakElecEnergy(W/sf),L_PeakGasEnergy(W),L_PeakGasEnergy(W/sf)
0,Elowe,08/02 09:00:00,E,Lowe,66528,629660,4.49436e+06,0,5.19055e+06,0,1441.82,6.8658,0,0
1,NWlowe,07/06 08:00:00,NW,Lowe,84672,428876,4.29955e+06,0,4.8131e+06,0,1336.97,6.36653,0,0
2,SWlowe,07/05 18:00:00,SW,Lowe,42336,557972,4.19014e+06,0,4.79045e+06,0,1330.68,6.33658,0,0


---
## Merge the 3 dataframes to perform delta peak from existing calculation


In [86]:
# merge existing and clear
MergedAnnualPeakElecData1 = pd.merge(allbaselinePeakDemandElec, allClearPeakDemandElec, left_index=True, right_index=True)

MergedAnnualPeakElecData1

,B_Scenario,B_Date_Time,B_Elevation,B_GlazingSystem,B_LightingEnergy[J](Hourly),B_FanEnergy[J](Hourly),B_CoolingEnergy[J](Hourly),B_HeatingEnergy[J](Hourly),B_PeakElecEnergy[J](Hourly),B_PeakGasEnergy[J](Hourly),...,C_LightingEnergy[J](Hourly),C_FanEnergy[J](Hourly),C_CoolingEnergy[J](Hourly),C_HeatingEnergy[J](Hourly),C_PeakElecEnergy[J](Hourly),C_PeakGasEnergy[J](Hourly),C_PeakElecEnergy(W),C_PeakElecEnergy(W/sf),C_PeakGasEnergy(W),C_PeakGasEnergy(W/sf)
0,Eexist,08/02 09:00:00,E,Exist,66528,670965,5.23197e+06,0,5.96946e+06,0,...,66528,664015,5.01435e+06,0,5.7449e+06,0,1595.8,7.59907,0,0
1,NWexist,07/05 17:00:00,NW,Exist,42336,657737,4.98142e+06,0,5.68149e+06,0,...,84672,461532,4.67848e+06,0,5.22469e+06,0,1451.3,6.91096,0,0
2,SWexist,07/05 19:00:00,SW,Exist,101304,735715,5.06429e+06,0,5.90131e+06,0,...,42336,636015,4.66139e+06,0,5.33974e+06,0,1483.26,7.06315,0,0


In [87]:
# # #merge the above with lowe
MergedAnnualPeakElecData2 = pd.merge(MergedAnnualPeakElecData1, allLowePeakDemandElec, left_index=True, right_index=True)

MergedAnnualPeakElecData2

,B_Scenario,B_Date_Time,B_Elevation,B_GlazingSystem,B_LightingEnergy[J](Hourly),B_FanEnergy[J](Hourly),B_CoolingEnergy[J](Hourly),B_HeatingEnergy[J](Hourly),B_PeakElecEnergy[J](Hourly),B_PeakGasEnergy[J](Hourly),...,L_LightingEnergy[J](Hourly),L_FanEnergy[J](Hourly),L_CoolingEnergy[J](Hourly),L_HeatingGasEnergy[J](Hourly),L_PeakElecEnergy[J](Hourly),L_PeakGasEnergy[J](Hourly),L_PeakElecEnergy(W),L_PeakElecEnergy(W/sf),L_PeakGasEnergy(W),L_PeakGasEnergy(W/sf)
0,Eexist,08/02 09:00:00,E,Exist,66528,670965,5.23197e+06,0,5.96946e+06,0,...,66528,629660,4.49436e+06,0,5.19055e+06,0,1441.82,6.8658,0,0
1,NWexist,07/05 17:00:00,NW,Exist,42336,657737,4.98142e+06,0,5.68149e+06,0,...,84672,428876,4.29955e+06,0,4.8131e+06,0,1336.97,6.36653,0,0
2,SWexist,07/05 19:00:00,SW,Exist,101304,735715,5.06429e+06,0,5.90131e+06,0,...,42336,557972,4.19014e+06,0,4.79045e+06,0,1330.68,6.33658,0,0


In [88]:
# # Jpin two dataframes in B
# PeakFinal = pd.merge(MergedAnnualPeakElecData1, MergedAnnualPeakElecData2,  how="inner")

# PeakFinal

In [89]:
list(MergedAnnualPeakElecData2.columns.values)

['B_Scenario',
 'B_Date_Time',
 'B_Elevation',
 'B_GlazingSystem',
 'B_LightingEnergy[J](Hourly)',
 'B_FanEnergy[J](Hourly)',
 'B_CoolingEnergy[J](Hourly)',
 'B_HeatingEnergy[J](Hourly)',
 'B_PeakElecEnergy[J](Hourly)',
 'B_PeakGasEnergy[J](Hourly)',
 'B_PeakElecEnergy(W)',
 'B_PeakElecEnergy(W/sf)',
 'B_PeakGasEnergy(W)',
 'B_PeakGasEnergy(W/sf)',
 'C_Scenario',
 'C_Date_Time',
 'C_Elevation',
 'C_GlazingSystem',
 'C_LightingEnergy[J](Hourly)',
 'C_FanEnergy[J](Hourly)',
 'C_CoolingEnergy[J](Hourly)',
 'C_HeatingEnergy[J](Hourly)',
 'C_PeakElecEnergy[J](Hourly)',
 'C_PeakGasEnergy[J](Hourly)',
 'C_PeakElecEnergy(W)',
 'C_PeakElecEnergy(W/sf)',
 'C_PeakGasEnergy(W)',
 'C_PeakGasEnergy(W/sf)',
 'L_Scenario',
 'L_Date_Time',
 'L_Elevation',
 'L_GlazingSystem',
 'L_LightingEnergy[J](Hourly)',
 'L_FanEnergy[J](Hourly)',
 'L_CoolingEnergy[J](Hourly)',
 'L_HeatingGasEnergy[J](Hourly)',
 'L_PeakElecEnergy[J](Hourly)',
 'L_PeakGasEnergy[J](Hourly)',
 'L_PeakElecEnergy(W)',
 'L_PeakElecEnergy(W

In [90]:
#Deltas of exist - clear

MergedAnnualPeakElecData2["DeltaClear_PeakElecEnergy(W)"] = MergedAnnualPeakElecData2["B_PeakElecEnergy(W)"] - MergedAnnualPeakElecData2["C_PeakElecEnergy(W)"]
MergedAnnualPeakElecData2["DeltaClear_PeakElecEnergy(W/sf)"] = MergedAnnualPeakElecData2["B_PeakElecEnergy(W/sf)"] - MergedAnnualPeakElecData2["C_PeakElecEnergy(W/sf)"]

MergedAnnualPeakElecData2
# AllMergedAnnualPeakData

,B_Scenario,B_Date_Time,B_Elevation,B_GlazingSystem,B_LightingEnergy[J](Hourly),B_FanEnergy[J](Hourly),B_CoolingEnergy[J](Hourly),B_HeatingEnergy[J](Hourly),B_PeakElecEnergy[J](Hourly),B_PeakGasEnergy[J](Hourly),...,L_CoolingEnergy[J](Hourly),L_HeatingGasEnergy[J](Hourly),L_PeakElecEnergy[J](Hourly),L_PeakGasEnergy[J](Hourly),L_PeakElecEnergy(W),L_PeakElecEnergy(W/sf),L_PeakGasEnergy(W),L_PeakGasEnergy(W/sf),DeltaClear_PeakElecEnergy(W),DeltaClear_PeakElecEnergy(W/sf)
0,Eexist,08/02 09:00:00,E,Exist,66528,670965,5.23197e+06,0,5.96946e+06,0,...,4.49436e+06,0,5.19055e+06,0,1441.82,6.8658,0,0,62.38,0.297048
1,NWexist,07/05 17:00:00,NW,Exist,42336,657737,4.98142e+06,0,5.68149e+06,0,...,4.29955e+06,0,4.8131e+06,0,1336.97,6.36653,0,0,126.89,0.604237
2,SWexist,07/05 19:00:00,SW,Exist,101304,735715,5.06429e+06,0,5.90131e+06,0,...,4.19014e+06,0,4.79045e+06,0,1330.68,6.33658,0,0,155.991,0.742816


In [91]:
# Deltas of exist - lowe

MergedAnnualPeakElecData2["DeltaLowe_PeakElecEnergy(W)"] = MergedAnnualPeakElecData2["B_PeakElecEnergy(W)"] - MergedAnnualPeakElecData2["L_PeakElecEnergy(W)"]
MergedAnnualPeakElecData2["DeltaLowe_PeakElecEnergy(W/sf)"] = MergedAnnualPeakElecData2["B_PeakElecEnergy(W/sf)"] - MergedAnnualPeakElecData2["L_PeakElecEnergy(W/sf)"]

MergedAnnualPeakElecData2

,B_Scenario,B_Date_Time,B_Elevation,B_GlazingSystem,B_LightingEnergy[J](Hourly),B_FanEnergy[J](Hourly),B_CoolingEnergy[J](Hourly),B_HeatingEnergy[J](Hourly),B_PeakElecEnergy[J](Hourly),B_PeakGasEnergy[J](Hourly),...,L_PeakElecEnergy[J](Hourly),L_PeakGasEnergy[J](Hourly),L_PeakElecEnergy(W),L_PeakElecEnergy(W/sf),L_PeakGasEnergy(W),L_PeakGasEnergy(W/sf),DeltaClear_PeakElecEnergy(W),DeltaClear_PeakElecEnergy(W/sf),DeltaLowe_PeakElecEnergy(W),DeltaLowe_PeakElecEnergy(W/sf)
0,Eexist,08/02 09:00:00,E,Exist,66528,670965,5.23197e+06,0,5.96946e+06,0,...,5.19055e+06,0,1441.82,6.8658,0,0,62.38,0.297048,216.367,1.03032
1,NWexist,07/05 17:00:00,NW,Exist,42336,657737,4.98142e+06,0,5.68149e+06,0,...,4.8131e+06,0,1336.97,6.36653,0,0,126.89,0.604237,241.22,1.14867
2,SWexist,07/05 19:00:00,SW,Exist,101304,735715,5.06429e+06,0,5.90131e+06,0,...,4.79045e+06,0,1330.68,6.33658,0,0,155.991,0.742816,308.571,1.46938


In [92]:
list(MergedAnnualPeakElecData2.columns.values)

['B_Scenario',
 'B_Date_Time',
 'B_Elevation',
 'B_GlazingSystem',
 'B_LightingEnergy[J](Hourly)',
 'B_FanEnergy[J](Hourly)',
 'B_CoolingEnergy[J](Hourly)',
 'B_HeatingEnergy[J](Hourly)',
 'B_PeakElecEnergy[J](Hourly)',
 'B_PeakGasEnergy[J](Hourly)',
 'B_PeakElecEnergy(W)',
 'B_PeakElecEnergy(W/sf)',
 'B_PeakGasEnergy(W)',
 'B_PeakGasEnergy(W/sf)',
 'C_Scenario',
 'C_Date_Time',
 'C_Elevation',
 'C_GlazingSystem',
 'C_LightingEnergy[J](Hourly)',
 'C_FanEnergy[J](Hourly)',
 'C_CoolingEnergy[J](Hourly)',
 'C_HeatingEnergy[J](Hourly)',
 'C_PeakElecEnergy[J](Hourly)',
 'C_PeakGasEnergy[J](Hourly)',
 'C_PeakElecEnergy(W)',
 'C_PeakElecEnergy(W/sf)',
 'C_PeakGasEnergy(W)',
 'C_PeakGasEnergy(W/sf)',
 'L_Scenario',
 'L_Date_Time',
 'L_Elevation',
 'L_GlazingSystem',
 'L_LightingEnergy[J](Hourly)',
 'L_FanEnergy[J](Hourly)',
 'L_CoolingEnergy[J](Hourly)',
 'L_HeatingGasEnergy[J](Hourly)',
 'L_PeakElecEnergy[J](Hourly)',
 'L_PeakGasEnergy[J](Hourly)',
 'L_PeakElecEnergy(W)',
 'L_PeakElecEnergy(W

In [93]:
# Export grouped file to csv
MergedAnnualPeakElecData2.to_csv("Scraper_Output/msp2_deltas_peakElec_com.csv", header=True, index=False)